In [ ]:
import torch
from models import black_box

# 设置工艺参数的上下限（顺序：P, v, t, h）
bounds = torch.tensor([
    [50, 200, 0.09, 0.1],   # 下限
    [150, 1000, 0.13, 0.4]  # 上限
], dtype=torch.double)

N_init = 16  # 初始点个数
X_init = torch.rand(N_init, 4) * (bounds[1] - bounds[0]) + bounds[0]
Y_init = black_box.func(X_init)  # 使用你定义的黑盒函数


In [ ]:
from optimization import qLogEHVI

slack=[0.01, -0.5, -0.5]
ref_point = qLogEHVI.get_ref_point(Y_init, slack)

In [25]:
from models.gp_model import build_model

model = build_model(X_init, Y_init)

slack = [-0.01, 0.5, 0.5]  # 每个目标的余量

ref_point = [
    Y_init[:, 0].min().item() - slack[0],  # Density（maximize）→ min 减
    Y_init[:, 1].max().item() + slack[1],  # Roughness（minimize）→ max 加
    Y_init[:, 2].max().item() + slack[2],  # Time（minimize）→ max 加
]

In [ ]:
from optimization.qLogEHVI import optimize_acq_fun

batch_size = 3
X_next, _ = optimize_acq_fun(model, Y_init, bounds, batch_size=batch_size, ref_point=ref_point)

In [27]:
Y_next = black_box.func(X_next)

X_all = torch.cat([X_init, X_next], dim=0)
Y_all = torch.cat([Y_init, Y_next], dim=0)

# 再次训练 surrogate model
model = build_model(X_all, Y_all)
